
# Model Finetuning Notebook

This notebook demonstrates the implementation and finetuning of various LLM models for the analysis of spurious correlations. 
Below are the steps involved in this project:
1. Data Loading and Preprocessing
2. Model Definition
3. Training and Validation
4. Hyperparameter Tuning and Optimization
5. Exporting Results

---


### Package dependencies (ignore if already installed)

In [ ]:
!pip install ipywidgets scikit-learn

In [ ]:
!pip install -U transformers
!pip install datasets

In [ ]:
!pip install huggingface-hub


In [ ]:
!pip install torch torchvision torchaudio


In [ ]:
!pip install flax
!pip install tensorflow

### Import statements and logging configuration

---

In [ ]:
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EvalPrediction,
    TFAutoModelForSequenceClassification,
    FlaxAutoModelForSequenceClassification
)
from typing import Dict, List, Tuple



import json
import logging
import numpy as np
import os
import shutil
import torch


# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Enable CUDA optimizations
torch.backends.cudnn.benchmark = True

---

### Huggingface setup
---

In [ ]:
os.environ["HF_TOKEN"] = None # insert HF token here if necessary
os.environ['FORCE_SAVE_BIN'] = '1'

In [ ]:
from huggingface_hub import login
login(token=None) # use token here if wanting to login


---

### Class definition with default dataset of toxic-spans
---

In [ ]:
class ToxicSpansAnalyzer:
    def __init__(self, model_name: str, dataset_name: str = 'heegyu/toxic-spans'):
        """
        Initialize the ToxicSpansAnalyzer with a specific model and dataset.
        """
        self.model_name = model_name
        self.dataset_name = dataset_name
        self.dataset = load_dataset(dataset_name)
        
        # Initialize tokenizer with optimized settings
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            use_fast=True,
            model_max_length=256
        )
        
        self.model = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        logger.info(f"Using device: {self.device}")
        
    def compute_metrics(self, eval_pred: EvalPrediction) -> Dict:
        """
        Compute evaluation metrics for the model.
        """
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        
        # Calculate metrics
        precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
        accuracy = accuracy_score(labels, predictions)
        
        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        }

    def preprocess_dataset(self) -> Dict:
        """
        Preprocess the dataset for training.
        """
        def preprocess_function(examples):
            return self.tokenizer(
                examples["text_of_post"],
                truncation=True,
                padding="max_length",
                max_length=256,
                return_attention_mask=True,
                return_tensors=None
            )
        
        def preprocess_labels(examples):
            examples["labels"] = examples["toxic"]
            return examples
        
        logger.info("Starting dataset preprocessing...")
        
        tokenized_datasets = self.dataset.map(
            preprocess_function,
            batched=True,
            batch_size=1000,
            num_proc=4
        )
        
        tokenized_datasets = tokenized_datasets.map(
            preprocess_labels,
            batched=True,
            num_proc=4
        )
        
        columns_to_remove = [
            "text_of_post", "toxic", "probability", "position",
            "type", "support", "position_probability"
        ]
        tokenized_datasets = tokenized_datasets.remove_columns(columns_to_remove)
        
        tokenized_datasets.set_format("torch")
        
        logger.info("Dataset preprocessing completed")
        return tokenized_datasets

    def save_models(self, output_dir: str, hyperparams: Dict):
        """
        Save the model in multiple formats (PyTorch, TensorFlow, and Flax) 
        along with hyperparameters and evaluation results.
        """
        logger.info(f"Saving models to {output_dir}")
        os.makedirs(output_dir, exist_ok=True)

        # Save hyperparameters and evaluation results
        hyperparams_path = os.path.join(output_dir, "hyperparameters.json")
        with open(hyperparams_path, 'w') as f:
            json.dump(hyperparams, f, indent=4)
        
        # Save evaluation results
        eval_results_path = os.path.join(output_dir, "eval_results.json")
        with open(eval_results_path, 'w') as f:
            json.dump(self.last_eval_results, f, indent=4)

        # Save PyTorch model
        pytorch_dir = os.path.join(output_dir, "pytorch")
        os.makedirs(pytorch_dir, exist_ok=True)
        # Save explicitly as a .bin file
        self.model = self.model.cpu()
        torch.save(self.model.state_dict(), os.path.join(output_dir, "pytorch_model.bin"))
        self.model.save_pretrained(pytorch_dir)

        # Save TensorFlow model
        try:
            tf_dir = os.path.join(output_dir, "tensorflow")
            os.makedirs(tf_dir, exist_ok=True)
            tf_model = TFAutoModelForSequenceClassification.from_pretrained(
                self.model_name, from_pt=True
            )
            tf_model.save_pretrained(tf_dir)
            logger.info("TensorFlow model saved successfully.")
        except Exception as e:
            logger.warning(f"Could not save TensorFlow model: {str(e)}")

        # Save Flax model
        try:
            flax_dir = os.path.join(output_dir, "flax")
            os.makedirs(flax_dir, exist_ok=True)
            flax_model = FlaxAutoModelForSequenceClassification.from_pretrained(
                self.model_name, from_pt=True
            )
            flax_model.save_pretrained(flax_dir)
            logger.info("Flax model saved successfully.")
        except Exception as e:
            logger.warning(f"Could not save Flax model: {str(e)}")

        logger.info("All models saved successfully.")

    def fine_tune(self, output_dir: str, hyperparams: Dict = None):
        """
        Fine-tune the model with hyperparameter configuration and save results.
        """
        if hyperparams is None:
            hyperparams = {'learning_rate': 2e-5, 'batch_size': 32, 'num_epochs': 3}
        
        tokenized_datasets = self.preprocess_dataset()
        
        # Initialize model
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=2
        ).to(self.device)
        
        training_args = TrainingArguments(
            output_dir=output_dir,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=hyperparams['learning_rate'],
            per_device_train_batch_size=hyperparams['batch_size'],
            per_device_eval_batch_size=hyperparams['batch_size'] * 2,
            num_train_epochs=hyperparams['num_epochs'],
            weight_decay=0.01,
            logging_dir=f"{output_dir}/logs",
            logging_steps=10,
            save_total_limit=2,
            fp16=torch.cuda.is_available(),
            gradient_checkpointing=True,
            dataloader_num_workers=4,
            dataloader_pin_memory=True,
            push_to_hub=False
        )
        
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=tokenized_datasets["train"],
            eval_dataset=tokenized_datasets["test"],
            tokenizer=self.tokenizer,
            compute_metrics=self.compute_metrics
        )
        
        # Train the model
        logger.info(f"Starting training with hyperparameters: {hyperparams}")
        train_result = trainer.train()
        
        # Evaluate the model
        logger.info("Evaluating model...")
        eval_results = trainer.evaluate()
        self.last_eval_results = eval_results
    
        return train_result, eval_results

---

### Hyperparameter tuning configuration
---

In [6]:
def hyperparameter_search(
    model_name: str,
    learning_rates: List[float],
    batch_sizes: List[int],
    base_output_dir: str
) -> Tuple[Dict, Dict]:
    """
    Perform a hyperparameter search to find the best configuration.
    """
    best_f1 = 0
    best_config = None
    best_results = None
    
    # Create a list to track all configurations and their performance
    all_configurations = []
    
    # Create output directory if it doesn't exist
    os.makedirs(base_output_dir, exist_ok=True)
    
    for lr in learning_rates:
        for bs in batch_sizes:
            logger.info(f"Testing learning rate: {lr}, batch size: {bs}")
            
            try:
                # Clear CUDA cache if available
                if torch.cuda.is_available():
                    torch.cuda.empty_cache()
                
                # Create analyzer and preprocess dataset
                analyzer = ToxicSpansAnalyzer(model_name)
                tokenized_datasets = analyzer.preprocess_dataset()
                
                # Prepare hyperparameters for this run
                hyperparams = {
                    'learning_rate': lr,
                    'batch_size': bs,
                    'num_epochs': 3,
                    'model_name': model_name
                }
                
                # Set output directory for this specific configuration
                output_dir = os.path.join(
                    base_output_dir, 
                    f"{model_name.replace('/', '_')}_lr{lr}_bs{bs}"
                )
                
                # Fine-tune and evaluate
                _, eval_results = analyzer.fine_tune(
                    output_dir=output_dir, 
                    hyperparams=hyperparams
                )
                
                # Extract F1 score
                f1_score = eval_results.get("eval_f1", 0)
                
                # Track all configurations
                configuration_result = {
                    'hyperparams': hyperparams,
                    'f1_score': f1_score,
                    'output_dir': output_dir
                }
                all_configurations.append(configuration_result)
                
                # Update best configuration if current is better
                if f1_score > best_f1:
                    best_f1 = f1_score
                    best_config = hyperparams
                    best_results = eval_results
                    
                    # Clean up previous best model directory
                    best_model_dir = os.path.join(base_output_dir, "best_model")
                    if os.path.exists(best_model_dir):
                        shutil.rmtree(best_model_dir)
                    
                    # Save the best model with multiple formats
                    os.makedirs(best_model_dir, exist_ok=True)
                    analyzer.save_models(best_model_dir, hyperparams)
                    
                    logger.info(f"New best model saved. F1 Score: {best_f1}")
            
            except Exception as e:
                logger.error(f"Error in hyperparameter search for {model_name} (LR:{lr}, BS:{bs}): {str(e)}")
    
    # Log and save all configurations for reference
    config_log_path = os.path.join(base_output_dir, "all_configurations.json")
    with open(config_log_path, 'w') as f:
        json.dump(all_configurations, f, indent=4)
    
    logger.info(f"Best F1 Score: {best_f1}")
    logger.info(f"Best Configuration: {best_config}")
    
    return best_config, best_results

---

### Upload to huggingface if necessary
---

In [7]:
from huggingface_hub import HfApi

In [8]:
def upload_model_to_huggingface(
    model_path: str, 
    repo_name: str, 
    username: str = None,  # Pass username directly
    organization: str = None, 
    private: bool = False
):
    """
    Upload a fine-tuned model to Hugging Face Model Hub.
    
    Args:
        model_path (str): Path to the local model directory
        repo_name (str): Name of the repository to create/update
        username (str, optional): Username for upload if not using organization
        organization (str, optional): Organization to upload under
        private (bool, optional): Whether the repository should be private. Defaults to False.
    """
    try:
        # Initialize Hugging Face API
        api = HfApi()
        
        # Determine the full repository name
        if username:
            full_repo_name = f"{username}/{repo_name}"
        else:
            # If no username or organization provided, raise an error
            raise ValueError("Must provide either username or organization")
        
        # Create the repository if it doesn't exist
        try:
            api.create_repo(
                repo_id=full_repo_name, 
                private=private,
                exist_ok=True  # Won't raise an error if repo already exists
            )
        except Exception as e:
            print(f"Repository creation/check failed: {e}")
        
        # Upload the entire model directory
        api.upload_folder(
            folder_path=model_path,
            repo_id=full_repo_name,
            commit_message="Upload fine-tuned toxic spans detection model"
        )
        
        print(f"Model successfully uploaded to {full_repo_name}")
        return full_repo_name
    
    except Exception as e:
        print(f"Error uploading model to Hugging Face: {e}")
        return None

---

### Main experiment loop
---

In [9]:
def run_experiment_with_hyperparameter_search(
    models: List[str],
    base_output_dir: str = "./results",
    learning_rates: List[float] = [1e-5, 2e-5, 3e-5, 5e-5],
    batch_sizes: List[int] = [8, 16, 32],
    upload_to_hub: bool = False,
    organization: str = None
):
    """
    Modified version of run_experiment_with_hyperparameter_search 
    that includes optional model hub upload.
    """
    results = {}
    
    for model_name in models:
        logger.info(f"Processing model: {model_name}")
        
        # Prepare model-specific output directory
        model_output_dir = os.path.join(base_output_dir, model_name.replace("/", "_"))
        
        try:
            # Perform hyperparameter search
            best_config, best_results = hyperparameter_search(
                model_name=model_name,
                learning_rates=learning_rates,
                batch_sizes=batch_sizes,
                base_output_dir=model_output_dir
            )
            
            # Store results
            results[model_name] = {
                "best_configuration": best_config,
                "best_results": best_results
            }
            
            # Optional: Upload to Hugging Face Model Hub
            if upload_to_hub:
                best_model_dir = os.path.join(base_output_dir, "best_model")
                
                # Create a descriptive repo name
                repo_name = f"toxic-spans-{model_name.replace('/', '-')}"
                
                # Upload the model
                uploaded_repo = upload_model_to_huggingface(
                    best_model_dir, 
                    repo_name, 
                    username='charleyisballer',
                    private=False  # Set to False if you want a public repo
                )
                
                # Add uploaded repo information to results
                if uploaded_repo:
                    results[model_name]["uploaded_repo"] = uploaded_repo
            
            logger.info(f"Best configuration for {model_name}: {best_config}")
        
        except Exception as e:
            logger.error(f"Error processing model {model_name}: {str(e)}")
            results[model_name] = {"error": str(e)}
    
    return results


In [ ]:
if __name__ == "__main__":
    # Define models and hyperparameters
    BERT_MODELS = [
        "lyeonii/bert-tiny",
        "lyeonii/bert-small",
        "lyeonii/bert-medium",
        "google-bert/bert-base-uncased",
        "google-bert/bert-large-uncased",
        "lyeonii/bert-mini"
    ]
    
    ROBERTA_MODELS = [
        "smallbenchnlp/roberta-small",
        "JackBAI/roberta-medium",
        "FacebookAI/roberta-base",
        "FacebookAI/roberta-large"
    ]

    # Set up base output directory
    base_output_dir = "./toxic-span-results"
    
    # Run experiments
    logger.info("Starting BERT experiments...")
    bert_results = run_experiment_with_hyperparameter_search(
        models=BERT_MODELS,
        base_output_dir=os.path.join(base_output_dir, "bert"),
        learning_rates=[1e-3, 1e-4, 1e-5, 1e-2],
        batch_sizes=[8, 16],
        upload_to_hub=True,
    )
    
    logger.info("Starting RoBERTa experiments...")
    roberta_results = run_experiment_with_hyperparameter_search(
        models=ROBERTA_MODELS,
        base_output_dir=os.path.join(base_output_dir, "roberta"),
        learning_rates=[1e-3, 1e-4, 1e-5, 1e-2],
        batch_sizes=[8, 16],
        upload_to_hub=True,
    )

    logger.info("Starting RoBERTa experiments...")
    roberta_results = run_experiment_with_hyperparameter_search(
        models=ROBERTA_MODELS,
        base_output_dir=os.path.join(base_output_dir, "roberta"),
        learning_rates=[1e-3, 1e-4, 1e-5, 1e-2],
        batch_sizes=[8, 16],
        upload_to_hub=True,
    )
    
    # Save overall results
    results_path = os.path.join(base_output_dir, "experiment_results.json")
    with open(results_path, 'w') as f:
        json.dump({
            "bert_results": bert_results,
            "roberta_results": roberta_results
        }, f, indent=4)
    
    logger.info("Hyperparameter search completed")